# Imports

In [133]:
import PCV
import wc1
import NecessarySets1 as ns
import PDACR11
import PDA11 as pda

In [134]:
class PushDownAutomaton:
    
    def __init__(self, symbols, end_sequence_symbol, acceptance_transition_name,
                 reject_transition_name, init_config, transition_table):
        self.symbols = symbols
        self.end_sequence_symbol = end_sequence_symbol
        self.acceptance_transition_name = acceptance_transition_name
        self.reject_transition_name = reject_transition_name
        self.transition_table = transition_table
        self.init_config = init_config
        self.stack = None
    
    def belongs_to_language(self, symbol_arr):
        if not self.symbols_in_language(symbol_arr):
            return False
        iterator = iter(symbol_arr)
        belongs = False
        input_symbol = next(iterator)
        self.stack = self.init_config.copy()
        while True:
            try:
                top_symbol = self.get_top_symbol()
                transition = self.transition_table.get_transition(top_symbol, input_symbol)
                if self.is_reject_transition(transition):
                    break
                if self.end_of_sequence(input_symbol) and self.is_acceptance_transition(transition):
                    belongs = True
                    break
                if transition.has_stack_operation():
                    self.stack = transition.stack_operation(self.stack)
                if transition.advance_input_operation:
                    input_symbol = next(iterator)
            except StopIteration:
                break
        return belongs
    
    def symbol_in_language(self, symbol):
        return symbol in self.symbols
    
    def symbols_in_language(self, symbol_arr):
        in_language = True
        for symbol in symbol_arr:
            if not self.symbol_in_language(symbol):
                in_language = False
                break
        return in_language
            
    def get_top_symbol(self):
        return self.stack[len(self.stack)-1]
                
    def end_of_sequence(self, symbol):
        return self.end_sequence_symbol == symbol
    
    def is_acceptance_transition(self, transition):
        return transition.name == self.acceptance_transition_name
    
    def is_reject_transition(self, transition):
        return transition.name == self.reject_transition_name

In [135]:
class PDACreator:
    def __init__(self, end_of_sequence_symbol, null_sequence_symbol, empty_PDA_symbol, initial_non_terminal,
                 non_terminals, productions, stack_operations, selection_set):
        self.end_of_sequence_symbol = end_of_sequence_symbol
        self.null_sequence_symbol = null_sequence_symbol
        self.empty_PDA_symbol = empty_PDA_symbol
        self.initial_non_terminal = initial_non_terminal
        self.non_terminals = non_terminals
        self.productions = productions
        self.stack_operations = stack_operations
        self.selection_set = selection_set
        
    def is_non_terminal(self, symbol):
        return symbol in self.non_terminals
        
    def is_terminal(self, symbol):
        return symbol != self.null_sequence_symbol and not self.is_non_terminal(symbol)
    
    def production_has_null_sequence_on_right_side(self, production):
        return production.right_side[0] == self.null_sequence_symbol
    
    def get_productions_same_non_terminal_l_side(self, non_terminal):
        productions_same_l_side = []
        for production in self.productions:
            if production.left_side == non_terminal:
                productions_same_l_side.append(production)
        return productions_same_l_side
    
    def get_productions_number_same_non_terminal_l_side(self, productions_same_l_side):
        productions_number_same_l_side = []
        for production in productions_same_l_side:
            productions_number_same_l_side.append(self.productions.index(production))
        return productions_number_same_l_side
    
    def get_productions_start_with_terminal(self, terminal, productions):
        productions_start_with_terminal = []
        for production in productions:
            if production.right_side[0] == terminal:
                productions_start_with_terminal.append(production)
        return productions_start_with_terminal
    
    def get_terminals_on_right_side(self, right_side):
        terminals = set()
        for symbol in right_side:
            if self.is_terminal(symbol):
                terminals.add(symbol)
        return terminals
    
    def get_productions_with_numbers(self, numbers):
        productions_with_numbers = []
        for number in numbers:
            productions_with_numbers.append(self.productions[number])
        return productions_with_numbers
    
    def production_starts_with_terminal(self, production):
        return self.is_terminal(production.right_side[0])
    
    def is_s_grammar(self):
        s_grammar = (0, None)
        for non_terminal in self.non_terminals:
            terminals = []
            productions_same_non_terminal_l_side = self.get_productions_same_non_terminal_l_side(non_terminal)
            for production in productions_same_non_terminal_l_side:
                if self.production_starts_with_terminal(production):
                    if not production.right_side[0] in terminals:
                        terminals.append(production.right_side[0])
                    else:
                        s_grammar = (1, self.get_productions_start_with_terminal(production.right_side[0],
                                                                                 productions_same_non_terminal_l_side))
                        # productions with same left side start with the the same terminal
                        break
                else:
                    if self.is_non_terminal(production.right_side[0]):
                        s_grammar = (2, production) # production starts with non_terminal
                        break
                    else:
                        s_grammar = (3, production) # production is nullable
                    break
        return s_grammar
    
    def selection_sets_are_disjoint(self, production_numbers):
        disjoint = True
        current_set = self.selection_set[production_numbers[0]]
        index = 1
        while(index < len(production_numbers)):
            current_set = current_set.intersection(self.selection_set[production_numbers[index]])
            if current_set != set():
                disjoint = False
                break
            index += 1
        return disjoint
    
    def get_selection_sets_not_disjoint(self, production_numbers):
        sets = set()
        for number in production_numbers:
            index = production_numbers.index(number)+1
            while(index < len(production_numbers)):
                current_set = self.selection_set[number]
                current_set = current_set.intersection(self.selection_set[production_numbers[index]])
                if current_set != set():
                    sets.add(number)
                    sets.add(production_numbers[index])
                index +=1
        return list(sets)
    
    def get_selection_sets_with_numbers(self, numbers):
        sets = []
        for number in numbers:
            sets.append(self.selection_set[number])
        return sets
    
    def is_q_grammar(self):
        q_grammar = (0, None)
        for non_terminal in self.non_terminals:
            terminals = []
            productions_same_non_terminal_l_side = self.get_productions_same_non_terminal_l_side(non_terminal)
            productions_number_same_l_side = self.get_productions_number_same_non_terminal_l_side(
            productions_same_non_terminal_l_side)
            if not self.selection_sets_are_disjoint(productions_number_same_l_side):
                numbers = self.get_selection_sets_not_disjoint(productions_number_same_l_side)
                productions_with_numbers = self.get_productions_with_numbers(numbers)
                selection_sets_with_numbers = self.get_selection_sets_with_numbers(numbers)
                q_grammar = (4, list(zip(productions_with_numbers, selection_sets_with_numbers))) # selection_sets are joint
                break
            for production in productions_same_non_terminal_l_side:
                if self.production_starts_with_terminal(production):
                    if not production.right_side[0] in terminals:
                        terminals.append(production.right_side[0])
                    else:
                        q_grammar = (1, self.get_productions_start_with_terminal(production.right_side[0],
                                                                                 productions_same_non_terminal_l_side))
                        break
                else:
                    if self.is_non_terminal(production.right_side[0]):
                        q_grammar = (2, production) # production starts with non_terminal
                        break
        return q_grammar
    
    def is_ll_grammar(self):
        ll_grammar = (0, None)
        for non_terminal in self.non_terminals:
            productions_same_non_terminal_l_side = self.get_productions_same_non_terminal_l_side(non_terminal)
            productions_number_same_l_side = self.get_productions_number_same_non_terminal_l_side(
            productions_same_non_terminal_l_side)
            if not self.selection_sets_are_disjoint(productions_number_same_l_side):
                numbers = self.get_selection_sets_not_disjoint(productions_number_same_l_side)
                productions_with_numbers = self.get_productions_with_numbers(numbers)
                selection_sets_with_numbers = self.get_selection_sets_with_numbers(numbers)
                ll_grammar = (4, list(zip(productions_with_numbers, selection_sets_with_numbers)))
                break
        return ll_grammar
    
    def get_terminals(self):
        terminals = []
        for production in self.productions:
            for symbol in production.right_side:
                if self.is_terminal(symbol) and not symbol in terminals:
                    terminals.append(symbol)
        return terminals
                    
    def get_input_symbols_PDA(self, terminals):
        input_symbols = terminals.copy()
        input_symbols.append(self.end_of_sequence_symbol)
        return input_symbols
    
    def get_initial_configuration_PDA(self):
        return [self.empty_PDA_symbol, self.initial_non_terminal]
    
    def production_terminal_alpha(self, production_same_l_side, input_symbol):
        terminal_alpha = None
        for production in production_same_l_side:
            if production.right_side[0] == input_symbol:
                terminal_alpha = production
        return terminal_alpha
    
    def get_transition_production_terminal_alpha(self, production, name):
        transition = None
        if len(production.right_side) >= 2:
            right_side = production.right_side[1:]
            right_side.reverse()
            transition = pda.Transition(name, self.stack_operations.replace_generator(right_side), True)
        else:
            transition = pda.Transition(name, self.stack_operations.pop_generator(), True)
        return transition
                
    def production_null_sequence_x_in_selection_p(self, production_numbers, symbol):
        x_in_selection_p = False
        for number in production_numbers:
            if symbol in list(self.selection_set[number]):
                x_in_selection_p = True
                break
        return x_in_selection_p
    
    def get_productions_type_beta(self, productions_same_l_side):
        productions_type_beta = []
        for production in productions_same_l_side:
            if self.is_non_terminal(production.right_side[0]):
                productions_type_beta.append(production)
        return productions_type_beta
        
    def get_transition_table_s_grammar(self, symbols_in_PDA, input_symbols_PDA):
        table = []
        n = 0
        for symbol in symbols_in_PDA:
            transitions = []
            for input_symbol in input_symbols_PDA:
                if self.is_non_terminal(symbol):
                    productions_same_l_side = self.get_productions_same_non_terminal_l_side(symbol)
                    production = self.production_terminal_alpha(productions_same_l_side, input_symbol)
                    if production:
                        transitions.append(self.get_transition_production_terminal_alpha(production, '#' + str(n)))
                        n+=1
                        continue
                elif self.is_terminal(symbol) and symbol == input_symbol:
                    transition = pda.Transition('#' + str(n), self.stack_operations.pop_generator(), True)
                    transitions.append(transition)
                    n += 1
                    continue
                elif symbol == self.empty_PDA_symbol and input_symbol == self.end_of_sequence_symbol:
                    transition = pda.Transition('A', None, None)
                    transitions.append(transition)
                    continue
                transition = pda.Transition('R', None, None)
                transitions.append(transition)
            table.append(transitions)
        return pda.TransitionTable(symbols_in_PDA, input_symbols_PDA, table)
    
    def production_null_sequence(self, productions):
        prod = None
        for production in productions:
            if self.production_has_null_sequence_on_right_side(production):
                prod = production
                break
        return prod
    
    def input_symbol_in_selection_set_of_production(self, input_symbol, production):
        return input_symbol in list(self.selection_set[self.productions.index(production)])

    def get_transition_table_q_grammar(self, symbols_in_PDA, input_symbols_PDA):
        table = []
        n = 0
        for symbol in symbols_in_PDA:
            transitions = []
            for input_symbol in input_symbols_PDA:
                if self.is_non_terminal(symbol):
                    productions_same_l_side = self.get_productions_same_non_terminal_l_side(symbol)
                    production = self.production_terminal_alpha(productions_same_l_side, input_symbol)
                    if production:
                        transitions.append(self.get_transition_production_terminal_alpha(production, '#' + str(n)))
                        n+=1
                        continue
                    production = self.production_null_sequence(productions_same_l_side)
                    if production and self.input_symbol_in_selection_set_of_production(input_symbol, production):
                        transition = pda.Transition('#' + str(n), self.stack_operations.pop_generator(), False)
                        transitions.append(transition)
                        n += 1
                        continue
                elif self.is_terminal(symbol) and symbol == input_symbol:
                    transition = pda.Transition('#' + str(n), self.stack_operations.pop_generator(), True)
                    transitions.append(transition)
                    n += 1
                    continue
                elif symbol == self.empty_PDA_symbol and input_symbol == self.end_of_sequence_symbol:
                    transition = pda.Transition('A', None, None)
                    transitions.append(transition)
                    continue
                transition = pda.Transition('R', None, None)
                transitions.append(transition)
            table.append(transitions)
        return pda.TransitionTable(symbols_in_PDA, input_symbols_PDA, table)
    
    def input_symbol_in_selection_set_of_production_type_beta(self,  input_symbol, productions_type_beta):
        production = None
        for production_beta in productions_type_beta:
            if self.input_symbol_in_selection_set_of_production(input_symbol, production_beta):
                production = production_beta
                break
        return production

    def get_transition_table_ll_grammar(self, symbols_in_PDA, input_symbols_PDA):
        table = []
        n = 0
        for symbol in symbols_in_PDA:
            transitions = []
            for input_symbol in input_symbols_PDA:
                if self.is_non_terminal(symbol):
                    productions_same_l_side = self.get_productions_same_non_terminal_l_side(symbol)
                    production = self.production_terminal_alpha(productions_same_l_side, input_symbol)
                    if production:
                        transitions.append(self.get_transition_production_terminal_alpha(production, '#' + str(n)))
                        n+=1
                        continue
                    productions_type_beta = self.get_productions_type_beta(productions_same_l_side)
                    production = self.input_symbol_in_selection_set_of_production_type_beta(input_symbol, productions_type_beta)
                    if productions_type_beta and production:
                        beta = production.right_side.copy()
                        beta.reverse()
                        transition = pda.Transition('#' + str(n), self.stack_operations.replace_generator(beta), False)
                        transitions.append(transition)
                        n += 1
                        continue
                    production = self.production_null_sequence(productions_same_l_side)
                    if production and self.input_symbol_in_selection_set_of_production(input_symbol, production):
                        transition = pda.Transition('#' + str(n), self.stack_operations.pop_generator(), False)
                        transitions.append(transition)
                        n += 1
                        continue
                elif self.is_terminal(symbol) and symbol == input_symbol:
                    transition = pda.Transition('#' + str(n), self.stack_operations.pop_generator(), True)
                    transitions.append(transition)
                    n += 1
                    continue
                elif symbol == self.empty_PDA_symbol and input_symbol == self.end_of_sequence_symbol:
                    transition = pda.Transition('A', None, None)
                    transitions.append(transition)
                    continue
                transition = pda.Transition('R', None, None)
                transitions.append(transition)
            table.append(transitions)
        return pda.TransitionTable(symbols_in_PDA, input_symbols_PDA, table)
    
    def is_production_terminal_alpha(self, production):
        return self.is_terminal(production.right_side[0])
    
    def is_production_beta(self, production):
        return self.is_non_terminal(production.right_side[0])
    
    def get_symbols_in_PDA_for_s_or_q(self, terminals):
        symbols_in_PDA = set(self.non_terminals)
        for terminal in terminals:
            for production in self.productions:
                if self.is_production_terminal_alpha(production) and len(production.right_side) >= 2:
                    if terminal in production.right_side[1:]:
                        symbols_in_PDA.add(terminal)
        symbols_in_PDA.add(self.empty_PDA_symbol)
        return list(symbols_in_PDA)
    
    def get_symbols_in_PDA_for_ll(self, terminals):
        symbols_in_PDA = set(self.non_terminals)
        for terminal in terminals:
            for production in self.productions:
                if self.is_production_terminal_alpha(production) and len(production.right_side) >= 2:
                    if terminal in production.right_side[1:]:
                        symbols_in_PDA.add(terminal)
                elif self.is_production_beta(production) and terminal in production.right_side:
                    symbols_in_PDA.add(terminal)
        symbols_in_PDA.add(self.empty_PDA_symbol)
        return list(symbols_in_PDA)
    
    def create_PDA(self):
        push_down_automata = None
        result = []
        terminals = self.get_terminals()
        input_symbols = self.get_input_symbols_PDA(terminals)
        initial_configuration = self.get_initial_configuration_PDA()
        s_grammar = self.is_s_grammar()
        q_grammar = self.is_q_grammar()
        ll_grammar = self.is_ll_grammar()
        
        if s_grammar[0] == 0:
            symbols_in_PDA = self.get_symbols_in_PDA_for_s_or_q(terminals)
            transition_table = self.get_transition_table_s_grammar(symbols_in_PDA, input_symbols)
            push_down_automata = pda.PushDownAutomaton(input_symbols, self.end_of_sequence_symbol, 'A', 'R',
                                                      initial_configuration, transition_table)
            result.append(('It is S grammar', push_down_automata, 0))
        elif s_grammar[0] == 1:
            result.append(('It is not S grammar, same non terminal starts with same terminal: ', s_grammar[1], 1))
        elif s_grammar[0] == 2:
            result.append(('It is not S grammar, production starts with a non terminal: ', s_grammar[1], 2))
        elif s_grammar[0] == 3:
            result.append(('It is not S grammar, production is nullable: ', s_grammar[1], 3))
            
        if q_grammar[0] == 0:
            symbols_in_PDA = self.get_symbols_in_PDA_for_s_or_q(terminals)
            transition_table = self.get_transition_table_q_grammar(symbols_in_PDA, input_symbols)
            if not push_down_automata:
                push_down_automata = pda.PushDownAutomaton(input_symbols, self.end_of_sequence_symbol, 'A', 'R',
                                                           initial_configuration, transition_table)
                result.append(('It is Q grammar', push_down_automata, 0))
            else:
                result.append(('It is Q grammar', None, 0))
        elif q_grammar[0] == 1:
            result.append(('It is not Q grammar, same non terminal starts with same terminal: ', q_grammar[1], 1))
        elif q_grammar[0] == 2:
            result.append(('It is not Q grammar, production starts with a non terminal: ', q_grammar[1], 2))
        elif q_grammar[0] == 4:
            result.append(('It is not Q grammar, selection sets of these productions are not disjoint: ', q_grammar[1], 4))

        if ll_grammar[0] == 0:
            symbols_in_PDA = self.get_symbols_in_PDA_for_ll(terminals)
            transition_table = self.get_transition_table_ll_grammar(symbols_in_PDA, input_symbols)
            if not push_down_automata:
                push_down_automata = pda.PushDownAutomaton(input_symbols, self.end_of_sequence_symbol, 'A', 'R',
                                                           initial_configuration, transition_table)
                result.append(('It is LL grammar', push_down_automata, 0))
            else:
                result.append(('It is LL grammar', None, 0))
            
        elif ll_grammar[0] == 4:
            result.append(('It is not LL grammar, selection sets of these productions are not disjoint: ', ll_grammar[1], 4))
        
        return result

In [136]:
s = [1, 2]
print(2 in s[1:])

True


# Variables

In [137]:
pcv_variables = {'string_separator': '\n', 'non_terminal_start': '<', 'non_terminal_finish': '>', 'arrow_tail': '-',
                 'arrow_head': '>', 'null_sequence_symbol': '%', 'empty_stack_symbol': '$', 'end_of_sequence_symbol': '!',
                 'non_terminal_pattern': '<[a-zA-Z]+>', 'arrow_op': '-{1}>{1}'}
pcv_variables['special_characters'] = [pcv_variables['arrow_tail'], pcv_variables['arrow_head'],
             pcv_variables['empty_stack_symbol'], pcv_variables['end_of_sequence_symbol'], pcv_variables['non_terminal_start'],
             pcv_variables['non_terminal_finish']]

ui_variables = {'w_title': 'Grammars', 'w_dimensions': '400x300', 'w_resizable': (False, False), 'txt_box_h': 25,
                'txt_box_w': 25, 'txt_box_coor': (0, 0), 'b_pda_name': 'Create Automata', 'c_b_name': 'Clear Grammar',
                'v_b_name': 'Validate String', 'b_width': 25, 'c_b_width': 25, 'e_width': 25, 'v_b_width': 25,
                'b_pda_coor': (225, 50), 'c_b_coor': (225, 100), 'e_coor': (225, 175), 'v_b_coor': (225, 225),
                'input_err_m': 'Your input does not follow the convention', 'err_0': 'Your productions are not well written',
                'err_1': 'There is an infinite cycle in your productions', 'err_2': 'The string must end with !',
                'err_3': 'These characters are reserved: ' + ''.join(pcv_variables['special_characters']),
                'err_4': 'You have not created an automata yet', 'err_5': 'This field cannot be empty',
                'err_6': 'The character !, goes at the end', 'label_1_txt': 'Enter a string to validate',
                'label_1_coor': (225, 150)}

# Application

In [138]:
class App:
    
    def __init__(self, pcv_variables, ui_variables):
        self.pcv_variables = pcv_variables
        self.ui_variables = ui_variables
        self.window = None
        self.txt_box = None
        self.b_pda = None
        self.clear_button = None
        self.production_creator = PCV.ProductionCreator(pcv_variables['string_separator'], pcv_variables['non_terminal_start'],
                                                       pcv_variables['non_terminal_finish'], pcv_variables['arrow_tail'],
                                                       pcv_variables['arrow_op'])
        self.production_validator = None
        self.production_set = None
        self.pda_creator = None
        self.push_down_automata = None
        self.entry = None
        self.validation_button = None
        self.string_to_validate = ''
        self.grammar = ''
        self.label_1 = None
        
    def start(self):
        self.window = wc1.create_main_window(self.ui_variables['w_title'], self.ui_variables['w_dimensions'],
                                       self.ui_variables['w_resizable'])
        self.txt_box = wc1.create_text_box(self.window, self.ui_variables['txt_box_h'], self.ui_variables['txt_box_w'])
        self.txt_box.place(x=self.ui_variables['txt_box_coor'][0], y=self.ui_variables['txt_box_coor'][1])
        self.b_pda = wc1.create_button(self.window, self.ui_variables['b_pda_name'], self.ui_variables['b_width'],
                                      self.create_push_down_automata)
        self.b_pda.place(x=self.ui_variables['b_pda_coor'][0], y=self.ui_variables['b_pda_coor'][1])
        self.clear_button = wc1.create_button(self.window, self.ui_variables['c_b_name'], self.ui_variables['c_b_width'],
                                      self.clear_grammars)
        self.clear_button.place(x=self.ui_variables['c_b_coor'][0], y=self.ui_variables['c_b_coor'][1])
        self.entry = wc1.create_entry(self.window, self.string_to_validate, self.ui_variables['e_width'])
        self.entry.place(x=self.ui_variables['e_coor'][0], y=self.ui_variables['e_coor'][1])
        self.validation_button = wc1.create_button(self.window, self.ui_variables['v_b_name'], self.ui_variables['v_b_width'],
                                      self.validate_string)
        self.validation_button.place(x=self.ui_variables['v_b_coor'][0], y=self.ui_variables['v_b_coor'][1])
        self.label_1 = wc1.create_label(self.window, self.ui_variables['label_1_txt'])
        self.label_1.place(x=self.ui_variables['label_1_coor'][0], y=self.ui_variables['label_1_coor'][1])
        self.window.mainloop()
        
    def valid_productions(self, productions):
        valid = True
        if not self.production_validator.productions_are_well_written(productions):
            valid = False
            wc1.show_error_message(self.ui_variables['err_0'])
        if self.production_validator.production_with_infinite_cycle(productions):
            valid = False
            wc1.show_error_message(self.ui_variables['err_1'])
        return valid
    
    def get_selection_set(self):
        nullable_non_terminals = self.production_set.get_nullable_non_terminals()
        nullable_productions = self.production_set.get_nullable_productions(nullable_non_terminals)
        set_of_firsts_for_non_terminals = self.production_set.get_set_of_firsts(nullable_non_terminals)
        set_of_firsts_for_productions = self.production_set.get_set_of_firsts_for_productions(nullable_non_terminals,
                                                                                set_of_firsts_for_non_terminals)
        set_of_nexts = self.production_set.replace_non_terminals(self.production_set.get_nexts_base_cases(
            nullable_non_terminals, set_of_firsts_for_non_terminals))
        selection_set = self.production_set.get_set_of_selection_for_productions(nullable_productions,
                                                                   set_of_firsts_for_productions, set_of_nexts)
        return selection_set
    
    def convert_set_to_string(self, _set):
        string = '{'
        for element in _set:
            if not _set.index(element) == len(_set)-1:
                string += (element + ', ')
            else:
                string += (element + '}')
        return string
            
    def obtain_pda(self):
        results = self.pda_creator.create_PDA()
        report = ''
        for result in results:
            if result[2] == 0:
                report += (result[0] +  '\n'+ '\n')
                if not self.push_down_automata and result[1]:
                    self.push_down_automata = result[1]
            elif result[2] == 1:
                report += (result[0] + '\n'+ '\n')
                for production in result[1]:
                    if not result[1].index(production) == len(result[1])-1:
                        report += (production.left_side + '->' + ''.join(production.right_side) + ', ')
                    else:
                        report += (production.left_side + '->' +''.join(production.right_side) + '\n'+ '\n')
            elif result[2] == 2 or result[2] == 3:
                report += (result[0] +  '\n'+ '\n')
                production = result[1]
                report += (production.left_side + '->' +''.join(production.right_side) + '\n'+ '\n')
            elif result[2] == 4:
                report += (result[0] + '\n'+ '\n')
                for pairs in result[1]:
                    production = pairs[0]
                    sets = list(pairs[1])
                    sets = self.convert_set_to_string(sets)
                    if not result[1].index(pairs) == len(result[1])-1:
                        report += ('production: ' + production.left_side + '->' + ''.join(production.right_side) +
                                   ' with selection set: '+ sets + '\n')
                    else:
                        report += ('production: ' + production.left_side + '->' + ''.join(production.right_side) +
                                   ' with selection set: '+ sets + '\n'+ '\n')                
        wc1.show_info_message(report)
        if self.push_down_automata:
            wc1.show_info_message('Automata Created')
            self.grammar = self.txt_box.get("1.0","end-1c")
        else:
            wc1.show_info_message('Automata was not Created')
    
    def clear_grammars(self):
        self.push_down_automata = None
        self.txt_box.delete("1.0","end-1c")
        self.grammar = ''
    
    def string_is_well_written(self, string):
        well_written = True
        try:
            if not string.index(self.pcv_variables['end_of_sequence_symbol']) == len(string)-1:
                wc1.show_error_message(self.ui_variables['err_6'])
                well_written = False
            else:
                for symbol in string:
                    if (symbol in self.pcv_variables['special_characters'] and
                        symbol != self.pcv_variables['end_of_sequence_symbol']):
                        wc1.show_error_message(self.ui_variables['err_3'])
                        well_written = False
                        break
        except:
            well_written = False
            wc1.show_error_message(self.ui_variables['err_2'])
        return  well_written
    
    def convert_string_to_list(self, string):
        _list = []
        for symbol in string:
            _list.append(symbol)
        return _list
    
    def validate_string(self):
        if self.push_down_automata:
            string = self.entry.get()
            if string:
                if self.string_is_well_written(string):
                    string_list = self.convert_string_to_list(string)
                    if self.push_down_automata.belongs_to_language(string_list):
                        wc1.show_info_message('The string: ' +  string + ' '+'belongs to the language generated by the grammar:' +
                                             '\n' + self.grammar)
                    else:
                        wc1.show_info_message('The string: ' +  string + ' '+
                                              'does not belong to the language generated by the grammar:' +
                                             '\n' + self.grammar)
            else:
                wc1.show_error_message(self.ui_variables['err_5'])
        else:
            wc1.show_error_message(self.ui_variables['err_4'])
        
    def create_push_down_automata(self):
        string = self.txt_box.get("1.0","end-1c")
        self.push_down_automata = None
        productions = self.production_creator.create_productions(self.production_creator.from_string_to_list(string))
        if not productions:
            wc1.show_error_message(self.ui_variables['input_err_m'])
        else:
            non_terminals = self.production_creator.get_non_terminals(productions)
            self.production_validator = PCV.ProductionValidator(non_terminals, self.pcv_variables['null_sequence_symbol'],
                                                               self.pcv_variables['non_terminal_pattern'],
                                                                self.pcv_variables['special_characters'])
            if self.valid_productions(productions):
                initial_non_terminal = non_terminals[0]
                self.production_set = ns.ProductionSet(initial_non_terminal, self.pcv_variables['null_sequence_symbol'],
                                                 self.pcv_variables['end_of_sequence_symbol'], productions, non_terminals)
                selection_set = self.get_selection_set()
                stack_operations = pda.StackOperation()
                self.pda_creator = PDACR11.PDACreator(self.pcv_variables['end_of_sequence_symbol'], 
                                                   self.pcv_variables['null_sequence_symbol'],
                                                   self.pcv_variables['empty_stack_symbol'],
                                                   initial_non_terminal, non_terminals, productions, stack_operations,
                                                   selection_set)
                self.obtain_pda()
            

# Start Application

In [139]:
app = App(pcv_variables, ui_variables)
app.start()

['b', 'a', 'p', 'q', 'd', 'r', 'f', 'g', '!']
<F>( R R R R #0 R #1 #2 #3 )
r( R R R R R #4 R R R )
$( R R R R R R R R A )
g( R R R R R R R #5 R )
b( #6 R R R R R R R R )
p( R R #7 R R R R R R )
<A>( #8 #9 #10 R #11 R #12 R #13 )
<rb>( R #14 R #15 #16 R #17 R #18 )
<D>( R R R R #19 #20 R R #21 )
<S>( #22 #23 R R R R R R R )
